In [2]:
from data import load_data
import matplotlib.pyplot as plt
import os
import numpy as np

import time

from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split

from keras.models import Model
from keras.layers.wrappers import TimeDistributed, Bidirectional
from keras.layers.recurrent import GRU
from keras.layers import Input
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding3D
from keras.layers.core import Lambda, Dropout, Flatten, Dense, Activation
from keras.optimizers import Adam
from keras import backend as K

CURRENT_PATH = '/home/ubuntu/assignments/machine-lip-reading/preprocessing'
DATA_PATH = CURRENT_PATH + '/../data'

def ctc_lambda_func(args):
    import tensorflow as tf
    y_pred, labels, input_length, label_length = args
    # From Keras example image_ocr.py:
    # the 2 is critical here since the first couple outputs of the RNN
    # tend to be garbage:
    # y_pred = y_pred[:, 2:, :]
    label_length = K.cast(tf.squeeze(label_length),'int32')
    input_length = K.cast(tf.squeeze(input_length),'int32')
    labels = K.ctc_label_dense_to_sparse(labels, label_length)
    #y_pred = y_pred[:, :, :]
    #return K.ctc_batch_cost(labels, y_pred, input_length, label_length, ignore_longer_outputs_than_inputs=True)
    return tf.nn.ctc_loss(labels, y_pred, input_length, ctc_merge_repeated=False,
                         ignore_longer_outputs_than_inputs = True, time_major = False)
def CTC(name, args):
	return Lambda(ctc_lambda_func, output_shape=(1,), name=name)(args)


def build_model(input_size, output_size = 28, max_string_len = 10):
    # model = Sequential()
    input_data = Input(name='the_input', shape=input_size, dtype='float32')
    x = ZeroPadding3D(padding=(0,2,2), name='padding1')(input_data)
    x = TimeDistributed(Conv2D(filters = 32, kernel_size = 5, strides = (2,2),
                             padding = 'same', activation = 'relu'))(x)
    print
    x = TimeDistributed(MaxPooling2D(pool_size=(2,2), strides=None, name='max1'))(x)
    x = Dropout(0.5)(x)

    x = TimeDistributed(Conv2D(filters=32, kernel_size=5, strides=(2, 2),
                               padding='same', activation='relu'))(x)
    x = TimeDistributed(MaxPooling2D(pool_size=(2,2), strides=None, name='max1'))(x)
    x = Dropout(0.5)(x)

    x = TimeDistributed(Conv2D(filters=4, kernel_size=5, strides=(2, 2),
                               padding='same', activation='relu'))(x)
    x = TimeDistributed(MaxPooling2D(pool_size=(2,2), strides=None, name='max1'))(x)
    x = Dropout(0.5)(x)

    input_lstm = TimeDistributed(Flatten())(x)

    x_lstm = Bidirectional(GRU(256, return_sequences=True, kernel_initializer='Orthogonal', name='gru1'), merge_mode='concat')(input_lstm)
    x_lstm = Dense(output_size, kernel_initializer='he_normal', name='dense1')(x_lstm)
    print("after dense1")
    y_pred = Activation('softmax', name='softmax')(x_lstm)

    labels = Input(name='the_labels', shape = [max_string_len], dtype='int32')
    input_length = Input(name = 'input_length', shape =[1], dtype = 'int32')
    label_length = Input(name = 'label_length', shape = [1], dtype = 'int32')
    loss = CTC('ctc',[y_pred, labels, input_length, label_length])
    model = Model(inputs=[input_data, labels, label_length, input_length],
                  outputs = loss)
    model.summary()
    # Build model here...

    return model
def pad_labels(labels, max_string_len):
    padding = np.ones((labels.shape[0], max_string_len - labels.shape[1])) * -1
    return np.concatenate((labels, padding), axis = 1)

def train(model, x_train, y_train, label_len_train, input_len_train, batch_size=256, epochs=100, val_train_ratio=0.2):
    max_string_len = 10
    if y_train.shape[1] != max_string_len:
        y_train = pad_labels(y_train, max_string_len)

    adam = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer=adam)
    history = model.fit(x = {'the_input':x_train, 'the_labels':y_train, 'label_length':label_len_train,
                             'input_length':input_len_train}, y = {'ctc': np.zeros([x_train.shape[0]])},
                        batch_size=batch_size,
                        epochs=epochs,
                        validation_split=val_train_ratio,
                        shuffle=True,
                        verbose=1)

    return history

def read_data():
    oh = OneHotEncoder()
    le = LabelEncoder()

    x = list()
    y = list()
    t = list()
    print("loading images...")
    for i, (img, words) in enumerate(load_data(DATA_PATH, verbose=False, framebyframe=False)):
        if img.shape[0] != 75:
            continue
        x.append(img)
        y.append(words)

        t += words.tolist()
        if i == 3:
            break

    t = le.fit_transform(t)
    oh.fit(t.reshape(-1, 1))

    print("convering to np array...")
    x = np.stack(x, axis=0)

    print("transforming y...")
    for i in range(len(y)):
        y_ = le.transform(y[i])
        y[i] = np.asarray(oh.transform(y_.reshape(-1, 1)).todense())
    y = np.stack(y, axis=0)

    return x, y

def main():
    epochs = 100

    start = time.time()
    print("loading data")
    x, y, label_len, input_len = load_data(DATA_PATH, verbose=True, num_samples=18, ctc_encoding=True)
    end = time.time()

    print("load data took", end-start)
    print("training data shapes:", x.shape, y.shape)
    x_train, x_test, y_train, y_test, label_len_train, label_len_test, \
    input_len_train, input_len_test = train_test_split(x, y, label_len, input_len, test_size=0.2)

    model = build_model(x.shape[1:], 28, max_string_len = 10)

    history = train(model, x_train, y_train, label_len_train, input_len_train, epochs=epochs)

    print("Saving model...")
    model.save('model.h5')

    # TODO: add visualization
#     print("Plotting...")
    #f, (ax1, ax2) = plt.subplots(2, 1)
    #ax1.plot(range(1, epochs+1), history.history['val_acc'], 'tab:blue', label="validation accuracy")
    #ax1.plot(range(1, epochs+1), history.history['acc'], 'tab:red', label="training accuracy")

    #ax2.plot(range(1, epochs+1), history.history['loss'], 'tab:orange', label="loss")
    #ax2.plot(range(1, epochs+1), history.history['val_loss'], 'tab:green', label="validation loss")

    #ax1.legend()
    #ax2.legend()

    #f.savefig('training.png', dpi=300)
    print("Done.")


if __name__ == "__main__":
    main()


loading data
reading: /home/ubuntu/assignments/machine-lip-reading/preprocessing/../data/s1/video/bbifzp.mpg
0: 16--21: bin
1: 21--26: blue
2: 26--28: in
3: 28--32: f
4: 32--40: zero
5: 40--51: please
reading: /home/ubuntu/assignments/machine-lip-reading/preprocessing/../data/s1/video/bbizzn.mpg
6: 16--22: bin
different size, skip
6: 22--27: blue
different size, skip
6: 27--31: in
different size, skip
6: 31--36: z
different size, skip
6: 36--45: zero
different size, skip
6: 45--51: now
different size, skip
reading: /home/ubuntu/assignments/machine-lip-reading/preprocessing/../data/s1/video/bbal8p.mpg
6: 22--27: bin
7: 27--31: blue
8: 31--32: at
9: 32--36: l
10: 36--40: eight
11: 40--50: please
reading: /home/ubuntu/assignments/machine-lip-reading/preprocessing/../data/s1/video/bbbf9a.mpg
12: 22--27: bin
13: 27--31: blue
14: 31--35: by
15: 35--39: f
16: 39--45: nine
17: 45--54: again
0: (11, 50, 100, 3)
Added: (11, 50, 100, 3)
1: (11, 50, 100, 3)
Added: (11, 50, 100, 3)
2: (11, 50, 100,

Epoch 19/100
11/11 [==============================] - 0s 29ms/step - loss: 14.9339 - val_loss: 9.5007
Epoch 20/100
11/11 [==============================] - 0s 29ms/step - loss: 14.9661 - val_loss: 9.4845
Epoch 21/100
11/11 [==============================] - 0s 29ms/step - loss: 14.9531 - val_loss: 9.4676
Epoch 22/100
11/11 [==============================] - 0s 29ms/step - loss: 14.9021 - val_loss: 9.4502
Epoch 23/100
11/11 [==============================] - 0s 28ms/step - loss: 14.8589 - val_loss: 9.4319
Epoch 24/100
11/11 [==============================] - 0s 29ms/step - loss: 14.8814 - val_loss: 9.4122
Epoch 25/100
11/11 [==============================] - 0s 28ms/step - loss: 14.8313 - val_loss: 9.3908
Epoch 26/100
11/11 [==============================] - 0s 29ms/step - loss: 14.8337 - val_loss: 9.3698
Epoch 27/100
11/11 [==============================] - 0s 29ms/step - loss: 14.7561 - val_loss: 9.3487
Epoch 28/100
11/11 [==============================] - 0s 28ms/step - loss: 14.6977

Epoch 100/100
11/11 [==============================] - 0s 29ms/step - loss: 13.7163 - val_loss: 8.5647
Saving model...
Done.


In [1]:
from data import load_data
CURRENT_PATH = '/home/ubuntu/assignments/machine-lip-reading/preprocessing'
DATA_PATH = CURRENT_PATH + '/../data'
x, y, label_len, input_len = load_data(DATA_PATH, verbose=True, num_samples=100, ctc_encoding=True)

('new speaker added: ', 's13')
reading: /home/ubuntu/assignments/machine-lip-reading/preprocessing/../data/s13/video/bbae2p.mpg
reading: /home/ubuntu/assignments/machine-lip-reading/preprocessing/../data/s13/video/bbaq9s.mpg
reading: /home/ubuntu/assignments/machine-lip-reading/preprocessing/../data/s13/video/bbaezn.mpg
reading: /home/ubuntu/assignments/machine-lip-reading/preprocessing/../data/s13/video/bbak4n.mpg
reading: /home/ubuntu/assignments/machine-lip-reading/preprocessing/../data/s13/video/bbae1s.mpg
reading: /home/ubuntu/assignments/machine-lip-reading/preprocessing/../data/s13/video/bbaq8n.mpg
reading: /home/ubuntu/assignments/machine-lip-reading/preprocessing/../data/s13/video/bbak7a.mpg
reading: /home/ubuntu/assignments/machine-lip-reading/preprocessing/../data/s13/video/bbak5s.mpg
reading: /home/ubuntu/assignments/machine-lip-reading/preprocessing/../data/s13/video/bbae3a.mpg
reading: /home/ubuntu/assignments/machine-lip-reading/preprocessing/../data/s13/video/bbak6p.mpg

In [3]:
from data import load_data
import numpy as np

CURRENT_PATH = '/home/ubuntu/assignments/machine-lip-reading/preprocessing'
DATA_PATH = CURRENT_PATH + '/../data'
SAVE_NUMPY_PATH = CURRENT_PATH + '/../data/numpy_results/'

speakers = load_data(DATA_PATH, verbose=True, num_samples=80, ctc_encoding=True)

new speaker added: s13
reading: /home/ubuntu/assignments/machine-lip-reading/preprocessing/../data/s13/video/bbae2p.mpg
reading: /home/ubuntu/assignments/machine-lip-reading/preprocessing/../data/s13/video/bbaq9s.mpg
reading: /home/ubuntu/assignments/machine-lip-reading/preprocessing/../data/s13/video/bbaezn.mpg
reading: /home/ubuntu/assignments/machine-lip-reading/preprocessing/../data/s13/video/bbak4n.mpg
reading: /home/ubuntu/assignments/machine-lip-reading/preprocessing/../data/s13/video/bbae1s.mpg
reading: /home/ubuntu/assignments/machine-lip-reading/preprocessing/../data/s13/video/bbaq8n.mpg
reading: /home/ubuntu/assignments/machine-lip-reading/preprocessing/../data/s13/video/bbak7a.mpg
reading: /home/ubuntu/assignments/machine-lip-reading/preprocessing/../data/s13/video/bbak5s.mpg
reading: /home/ubuntu/assignments/machine-lip-reading/preprocessing/../data/s13/video/bbae3a.mpg
reading: /home/ubuntu/assignments/machine-lip-reading/preprocessing/../data/s13/video/bbak6p.mpg
new spe

In [7]:
for speaker in speakers:
    print("hey let's read the numpy results for speaker " + speaker)
    print("reading x_raw: ")
    x_raw = np.load(SAVE_NUMPY_PATH + speaker + "_x.npy")
    print(x_raw.shape)
    print("reading y_raw: ")
    y_raw = np.load(SAVE_NUMPY_PATH + speaker + "_y.npy")
    print(y_raw.shape)
    print("reading word_len_list: ")
    word_len_list = np.load(SAVE_NUMPY_PATH + speaker + "_word_len_list.npy")
    print(word_len_list.shape)
    print("reading input_len_list: ")
    input_len_list = np.load(SAVE_NUMPY_PATH + speaker + "_input_len_list.npy")
    print(input_len_list.shape)

hey let's read the numpy results for speaker s13
reading x_raw: 
(60,)
reading y_raw: 
(60,)
reading word_len_list: 
(60,)
reading input_len_list: 
(60,)
hey let's read the numpy results for speaker s1
reading x_raw: 
(20,)
reading y_raw: 
(20,)
reading word_len_list: 
(20,)
reading input_len_list: 
(20,)


In [ ]:
import numpy as np
import os
from align import read_align
from video import read_video
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import re

CURRENT_PATH = os.path.dirname(os.path.abspath(__file__))
DATA_PATH = CURRENT_PATH + '/../data'
PREDICTOR_PATH = CURRENT_PATH + '/shape_predictor_68_face_landmarks.dat'
SAVE_NUMPY_PATH = CURRENT_PATH + '/../data/numpy_results'


def text_to_labels(text):
    ret = []
    for char in text:
        if char >= 'a' and char <= 'z':
            ret.append(ord(char) - ord('a'))
        elif char == ' ':
            ret.append(26)
    return ret

def labels_to_text(labels):
# 26 is space, 27 is CTC blank char
    text = ''
    for c in labels:
        if c >= 0 and c < 26:
            text += chr(c + ord('a'))
        elif c == 26:
            text += ' '
    return text

def load_data(datapath, verbose=False, num_samples=-1, ctc_encoding=False):
    oh = OneHotEncoder()
    le = LabelEncoder()

    counter = 0
    done = False

    max_len = 0
    max_word_len = 0

    word_len_list = []
    input_len_list = []

    x_raw = list()
    y_raw = list()
    
    pattern = re.compile("s[0-9]") 
    speakers = []
    x_raws = {}
    y_raws = {}
    word_len_lists = {}
    input_len_lists = {}
    prev_speaker_count = 0
    
    for root, dirs, files in os.walk(datapath):
        check = root.split("/")[-1]
        match = pattern.findall(check)
        if (len(match) > 0):
            if check.index(match[0]) == 0 and check not in speakers:
#                 if len(speakers) > 0:
#                     x_raws.append(x_raw[prev_speaker_count : counter])
#                     prev_speaker_count = counter
#                     if verbose:
#                         print("previous speaker x_raws")
#                         print(x_raws)               
                speakers.append(check)
                x_raws[check] = []
                y_raws[check] = []
                if verbose:
                    print("new speaker added")
                    print(check, match[0])             
        
        for name in files:
            if '.mpg' in name:
                if verbose is True:
                    print("reading: " + root + "/" + name)

                video = read_video(os.path.join(root, name), PREDICTOR_PATH)
                alignments = read_align(os.path.join(root, '../align/', name.split(".")[0] + ".align"))
                
                for start, stop, word in alignments:
                    if word == 'sil' or word == 'sp':
                        continue
                    
                    if verbose is True:
                        print(str(counter) + ": " + str(start) + "--" + str(stop) + ": " + word)
                    
                    _, d1, d2, d3 = video[start:stop].shape
                    
                    if (len(x_raw) > 0):
                        _, prev_d1, prev_d2, prev_d3 = x_raw[-1].shape
                        if (d1, d2, d3) != (prev_d1, prev_d2, prev_d3):
                            if verbose is True:
                                print("different size, skip")
                            continue
                    
                    x_raw.append(video[start:stop])
                    y_raw.append(word)
                    temp = video[start:stop]
                    x_raws[speakers[-1]].append(temp)
                    y_raws[speakers[-1]].append(word)
                    
                    max_word_len = max(max_word_len, len(word))
                    max_len = max(max_len, stop-start)
                    word_len_list.append(len(word))
                    input_len_list.append(stop-start)
                    word_len_lists[speakers[-1]].append(len(word))
                    input_len_lists[speakers[-1]].append(stop-start)
                    counter += 1
                    if counter == num_samples:
                        done = True
                        break
            
            if done:
                break
        
        if done:
            break
    
    if not ctc_encoding:
        y_raw = le.fit_transform(y_raw)
#         y = oh.fit_transform(y_raw.reshape(-1, 1)).todense()


    for i in range(len(x_raw)):
        result = np.zeros((max_len, x_raw[i].shape[1], x_raw[i].shape[2], x_raw[i].shape[3]))
        result[:x_raw[i].shape[0], :x_raw[i].shape[1], :x_raw[i].shape[2], :x_raw[i].shape[3]] = x_raw[i]
        if verbose is True:
            print(str(i) + ": " + str(result.shape))
        x_raw[i] = result

        if verbose is True:
            print("Added: " + str(x_raw[i].shape)) 
        
        if ctc_encoding:
            res = np.ones(max_word_len) * -1
            enc = np.array(text_to_labels(y_raw[i]))
            res[:enc.shape[0]] = enc
            y_raw[i] = res

    for speaker in speakers:
        np_save = SAVE_NUMPY_PATH + "/" + speaker
        np.save(np_save + "_x", x_raws[speaker])
        np.save(np_save + "_y", y_raws[speaker])
        np.save(np_save + "_word_len_list", np.array(word_len_lists[speaker]))
        np.save(np_save + "_input_len_list", np.array(input_len_lists[speaker]))
        
#     if ctc_encoding:
#         y = np.stack(y_raw, axis=0)

#     x = np.stack(x_raw, axis=0)

#     print(root)
#     np.save("test_savex", x_raw)
#     np.save("test_savey", y_raw)
#     return x, y, np.array(word_len_list), np.array(input_len_list)
    return speakers #, np.array(word_len_list), np.array(input_len_list)

if __name__ == "__main__":
    X, y = load_data(DATA_PATH, verbose=True, ctc_encoding=True, num_samples=15)
    print("X:", X.shape)
    print("y:", y.shape)
    print(y)
